In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

# ---------------------
# 1. 데이터 로드 (앞선 분석 결과 활용)
# ---------------------
orders = pd.read_csv("olist_orders_dataset.csv", parse_dates=["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"])
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")

order_df = orders.merge(reviews, on="order_id", how="left").merge(customers, on="customer_id", how="left")

order_df["delivery_days"] = (order_df["order_delivered_customer_date"] - order_df["order_purchase_timestamp"]).dt.days
order_df["delay_vs_estimate_days"] = (order_df["order_delivered_customer_date"] - order_df["order_estimated_delivery_date"]).dt.days
order_df["on_time"] = order_df["delay_vs_estimate_days"] <= 0

# ---------------------
# 2. KPI 집계
# ---------------------
orders_total = len(order_df)
mean_delivery_days = order_df["delivery_days"].mean()
median_delivery_days = order_df["delivery_days"].median()
on_time_rate = order_df["on_time"].mean()

corr = order_df[["delay_vs_estimate_days","review_score"]].corr().iloc[0,1]
x = order_df["delay_vs_estimate_days"].dropna()
y = order_df.loc[x.index, "review_score"]
a, b = np.polyfit(x, y, 1)

state_perf = order_df.groupby("customer_state").agg(
    orders=("order_id","count"),
    mean_delivery_days=("delivery_days","mean"),
    on_time_rate=("on_time","mean"),
    avg_review_score=("review_score","mean")
).reset_index()

# ---------------------
# 3. 시각화 이미지 저장
# ---------------------
plt.figure(figsize=(8,5))
order_df["delivery_days"].hist(bins=50)
plt.title("배송 소요일 분포")
plt.xlabel("일수")
plt.ylabel("주문 수")
plt.savefig("delivery_distribution.png")
plt.close()

plt.figure(figsize=(8,5))
plt.scatter(order_df["delay_vs_estimate_days"], order_df["review_score"], alpha=0.3)
plt.title("배송 지연일수 vs 리뷰 점수")
plt.xlabel("지연일수")
plt.ylabel("리뷰점수")
plt.savefig("delay_vs_review.png")
plt.close()

# ---------------------
# 4. PDF 리포트 생성
# ---------------------
doc = SimpleDocTemplate("shipping_performance_report.pdf")
styles = getSampleStyleSheet()
flow = []

flow.append(Paragraph("<b>배송 성과 분석 리포트</b>", styles['Title']))
flow.append(Spacer(1,12))

# KPI 요약
kpi_text = f"""
총 주문 수: {orders_total:,}<br/>
평균 배송일수: {mean_delivery_days:.2f}일<br/>
중앙값 배송일수: {median_delivery_days:.2f}일<br/>
정시배송률: {on_time_rate*100:.2f}%<br/>
지연일수 vs 리뷰점수 상관계수: {corr:.3f}<br/>
회귀식: 리뷰 = {a:.3f} * 지연일수 + {b:.3f}
"""
flow.append(Paragraph(kpi_text, styles['Normal']))
flow.append(Spacer(1,12))

# 주별 성과 테이블 (상위 5)
state_table = state_perf.sort_values("mean_delivery_days", ascending=False).head(5)
data = [["주(State)", "주문수", "평균 배송일", "정시율", "평균 리뷰"]] + state_table.values.tolist()
t = Table(data)
t.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
    ('GRID', (0,0), (-1,-1), 0.5, colors.black)
]))
flow.append(Paragraph("주별 배송 성과 (느린 Top 5)", styles['Heading2']))
flow.append(t)
flow.append(Spacer(1,12))

# 시각화 삽입
flow.append(Paragraph("배송 소요일 분포", styles['Heading2']))
flow.append(Image("delivery_distribution.png", width=400, height=250))
flow.append(Spacer(1,12))

flow.append(Paragraph("배송 지연 vs 리뷰", styles['Heading2']))
flow.append(Image("delay_vs_review.png", width=400, height=250))

# PDF 저장
doc.build(flow)
print("PDF 리포트가 생성되었습니다: shipping_performance_report.pdf")


ModuleNotFoundError: No module named 'reportlab'

In [2]:
pip install python-pptx

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----------------------- ---------------- 2.4/4.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 14.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install reportlab

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 13.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

# ---------------------
# pptx 모듈 임포트 실패 시 안내 메시지 처리
# ---------------------
try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
    pptx_available = True
except ModuleNotFoundError:
    print("⚠️ python-pptx 라이브러리가 설치되어 있지 않습니다. 설치하려면 'pip install python-pptx'를 실행하세요.")
    pptx_available = False

# ---------------------
# 1. 데이터 로드
# ---------------------
orders = pd.read_csv("olist_orders_dataset.csv", parse_dates=["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"])
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")

order_df = orders.merge(reviews, on="order_id", how="left").merge(customers, on="customer_id", how="left")

order_df["delivery_days"] = (order_df["order_delivered_customer_date"] - order_df["order_purchase_timestamp"]).dt.days
order_df["delay_vs_estimate_days"] = (order_df["order_delivered_customer_date"] - order_df["order_estimated_delivery_date"]).dt.days
order_df["on_time"] = order_df["delay_vs_estimate_days"] <= 0

# ---------------------
# 2. KPI 집계
# ---------------------
orders_total = len(order_df)
mean_delivery_days = order_df["delivery_days"].mean()
median_delivery_days = order_df["delivery_days"].median()
on_time_rate = order_df["on_time"].mean()

corr = order_df[["delay_vs_estimate_days","review_score"]].corr().iloc[0,1]
x = order_df["delay_vs_estimate_days"].dropna()
y = order_df.loc[x.index, "review_score"]
a, b = np.polyfit(x, y, 1)

state_perf = order_df.groupby("customer_state").agg(
    orders=("order_id","count"),
    mean_delivery_days=("delivery_days","mean"),
    on_time_rate=("on_time","mean"),
    avg_review_score=("review_score","mean")
).reset_index()

# ---------------------
# 3. 시각화 이미지 저장
# ---------------------
plt.figure(figsize=(8,5))
order_df["delivery_days"].hist(bins=50)
plt.title("배송 소요일 분포")
plt.xlabel("일수")
plt.ylabel("주문 수")
plt.savefig("delivery_distribution.png")
plt.close()

plt.figure(figsize=(8,5))
plt.scatter(order_df["delay_vs_estimate_days"], order_df["review_score"], alpha=0.3)
plt.title("배송 지연일수 vs 리뷰 점수")
plt.xlabel("지연일수")
plt.ylabel("리뷰점수")
plt.savefig("delay_vs_review.png")
plt.close()

# ---------------------
# 4. PDF 리포트 생성
# ---------------------
doc = SimpleDocTemplate("shipping_performance_report.pdf")
styles = getSampleStyleSheet()
flow = []

flow.append(Paragraph("<b>배송 성과 분석 리포트</b>", styles['Title']))
flow.append(Spacer(1,12))

kpi_text = f"""
총 주문 수: {orders_total:,}<br/>
평균 배송일수: {mean_delivery_days:.2f}일<br/>
중앙값 배송일수: {median_delivery_days:.2f}일<br/>
정시배송률: {on_time_rate*100:.2f}%<br/>
지연일수 vs 리뷰점수 상관계수: {corr:.3f}<br/>
회귀식: 리뷰 = {a:.3f} * 지연일수 + {b:.3f}
"""
flow.append(Paragraph(kpi_text, styles['Normal']))
flow.append(Spacer(1,12))

state_table = state_perf.sort_values("mean_delivery_days", ascending=False).head(5)
data = [["주(State)", "주문수", "평균 배송일", "정시율", "평균 리뷰"]] + state_table.values.tolist()
t = Table(data)
t.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
    ('GRID', (0,0), (-1,-1), 0.5, colors.black)
]))
flow.append(Paragraph("주별 배송 성과 (느린 Top 5)", styles['Heading2']))
flow.append(t)
flow.append(Spacer(1,12))

flow.append(Paragraph("배송 소요일 분포", styles['Heading2']))
flow.append(Image("delivery_distribution.png", width=400, height=250))
flow.append(Spacer(1,12))

flow.append(Paragraph("배송 지연 vs 리뷰", styles['Heading2']))
flow.append(Image("delay_vs_review.png", width=400, height=250))

doc.build(flow)
print("PDF 리포트가 생성되었습니다: shipping_performance_report.pdf")

# ---------------------
# 5. PPTX 슬라이드 생성 (pptx 설치된 경우만 실행)
# ---------------------
if pptx_available:
    prs = Presentation()

    # 타이틀 슬라이드
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = "배송 성과 분석"
    subtitle.text = "Olist 데이터 기반 경영 보고"

    # KPI 슬라이드
    bullet_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]

    title_shape.text = "핵심 KPI"
    text_frame = body_shape.text_frame
    text_frame.text = f"총 주문 수: {orders_total:,}\n평균 배송일수: {mean_delivery_days:.2f}일\n중앙값 배송일수: {median_delivery_days:.2f}일\n정시배송률: {on_time_rate*100:.2f}%"

    # 리뷰 영향 슬라이드
    slide = prs.slides.add_slide(bullet_slide_layout)
    slide.shapes.title.text = "배송 지연과 리뷰의 관계"
    body_shape = slide.placeholders[1]
    text_frame = body_shape.text_frame
    text_frame.text = f"상관계수: {corr:.3f}\n회귀식: 리뷰 = {a:.3f} * 지연일수 + {b:.3f}"
    slide.shapes.add_picture("delay_vs_review.png", Inches(4), Inches(2), width=Inches(5), height=Inches(3))

    # 주별 성과 슬라이드
    slide = prs.slides.add_slide(bullet_slide_layout)
    slide.shapes.title.text = "주별 배송 성과 (느린 Top 5)"
    rows, cols = state_table.shape

    left = Inches(0.5)
    top = Inches(2.0)
    width = Inches(9)
    height = Inches(1.0)

    table = slide.shapes.add_table(rows+1, cols, left, top, width, height).table

    # 헤더 채우기
    for j, col_name in enumerate(state_table.columns):
        table.cell(0, j).text = str(col_name)

    # 데이터 채우기
    for i in range(rows):
        for j in range(cols):
            table.cell(i+1, j).text = str(round(state_table.iloc[i, j],2))

    # 배송 소요일 분포 슬라이드
    slide = prs.slides.add_slide(bullet_slide_layout)
    slide.shapes.title.text = "배송 소요일 분포"
    slide.shapes.add_picture("delivery_distribution.png", Inches(2), Inches(2), width=Inches(6), height=Inches(4))

    prs.save("shipping_performance_presentation.pptx")
    print("PPTX 프레젠테이션이 생성되었습니다: shipping_performance_presentation.pptx")


C:\Users\user\AppData\Local\Temp\ipykernel_27188\1642587043.py:60: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1642587043.py:60: UserWarning: Glyph 47928 (\N{HANGUL SYLLABLE MUN}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1642587043.py:60: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1642587043.py:60: UserWarning: Glyph 48176 (\N{HANGUL SYLLABLE BAE}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1642587043.py:60: UserWarning: Glyph 49569 (\N{HANGUL SYLLABLE SONG}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Loca

PDF 리포트가 생성되었습니다: shipping_performance_report.pdf


TypeError: type str doesn't define __round__ method

In [7]:
import os
os.getcwd()

'c:\\Users\\user\\pjt-data-analysis'

In [8]:
import os
os.startfile("shipping_performance_report.pdf")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------------------
# reportlab 모듈 임포트 실패 시 안내 메시지 처리
# ---------------------
try:
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
    from reportlab.lib.styles import getSampleStyleSheet
    from reportlab.lib import colors
    reportlab_available = True
except ModuleNotFoundError:
    print("⚠️ reportlab 라이브러리가 설치되어 있지 않습니다. 설치하려면 'pip install reportlab'를 실행하세요.")
    reportlab_available = False

# ---------------------
# pptx 모듈 임포트 실패 시 안내 메시지 처리
# ---------------------
try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
    pptx_available = True
except ModuleNotFoundError:
    print("⚠️ python-pptx 라이브러리가 설치되어 있지 않습니다. 설치하려면 'pip install python-pptx'를 실행하세요.")
    pptx_available = False

# ---------------------
# 1. 데이터 로드
# ---------------------
orders = pd.read_csv("olist_orders_dataset.csv", parse_dates=["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"])
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")

order_df = orders.merge(reviews, on="order_id", how="left").merge(customers, on="customer_id", how="left")

order_df["delivery_days"] = (order_df["order_delivered_customer_date"] - order_df["order_purchase_timestamp"]).dt.days
order_df["delay_vs_estimate_days"] = (order_df["order_delivered_customer_date"] - order_df["order_estimated_delivery_date"]).dt.days
order_df["on_time"] = order_df["delay_vs_estimate_days"] <= 0

# ---------------------
# 2. KPI 집계
# ---------------------
orders_total = len(order_df)
mean_delivery_days = order_df["delivery_days"].mean()
median_delivery_days = order_df["delivery_days"].median()
on_time_rate = order_df["on_time"].mean()

corr = order_df[["delay_vs_estimate_days","review_score"]].corr().iloc[0,1]
x = order_df["delay_vs_estimate_days"].dropna()
y = order_df.loc[x.index, "review_score"]
a, b = np.polyfit(x, y, 1)

state_perf = order_df.groupby("customer_state").agg(
    orders=("order_id","count"),
    mean_delivery_days=("delivery_days","mean"),
    on_time_rate=("on_time","mean"),
    avg_review_score=("review_score","mean")
).reset_index()

# ---------------------
# 3. 시각화 이미지 저장
# ---------------------
plt.figure(figsize=(8,5))
order_df["delivery_days"].hist(bins=50)
plt.title("배송 소요일 분포")
plt.xlabel("일수")
plt.ylabel("주문 수")
plt.savefig("delivery_distribution.png")
plt.close()

plt.figure(figsize=(8,5))
plt.scatter(order_df["delay_vs_estimate_days"], order_df["review_score"], alpha=0.3)
plt.title("배송 지연일수 vs 리뷰 점수")
plt.xlabel("지연일수")
plt.ylabel("리뷰점수")
plt.savefig("delay_vs_review.png")
plt.close()

# ---------------------
# 4. PDF 리포트 생성 (reportlab 설치된 경우만)
# ---------------------
if reportlab_available:
    doc = SimpleDocTemplate("shipping_performance_report.pdf")
    styles = getSampleStyleSheet()
    flow = []

    flow.append(Paragraph("<b>배송 성과 분석 리포트</b>", styles['Title']))
    flow.append(Spacer(1,12))

    kpi_text = f"""
    총 주문 수: {orders_total:,}<br/>
    평균 배송일수: {mean_delivery_days:.2f}일<br/>
    중앙값 배송일수: {median_delivery_days:.2f}일<br/>
    정시배송률: {on_time_rate*100:.2f}%<br/>
    지연일수 vs 리뷰점수 상관계수: {corr:.3f}<br/>
    회귀식: 리뷰 = {a:.3f} * 지연일수 + {b:.3f}
    """
    flow.append(Paragraph(kpi_text, styles['Normal']))
    flow.append(Spacer(1,12))

    state_table = state_perf.sort_values("mean_delivery_days", ascending=False).head(5)
    data = [["주(State)", "주문수", "평균 배송일", "정시율", "평균 리뷰"]] + state_table.values.tolist()
    t = Table(data)
    t.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
        ('GRID', (0,0), (-1,-1), 0.5, colors.black)
    ]))
    flow.append(Paragraph("주별 배송 성과 (느린 Top 5)", styles['Heading2']))
    flow.append(t)
    flow.append(Spacer(1,12))

    flow.append(Paragraph("배송 소요일 분포", styles['Heading2']))
    flow.append(Image("delivery_distribution.png", width=400, height=250))
    flow.append(Spacer(1,12))

    flow.append(Paragraph("배송 지연 vs 리뷰", styles['Heading2']))
    flow.append(Image("delay_vs_review.png", width=400, height=250))

    doc.build(flow)
    print("PDF 리포트가 생성되었습니다: shipping_performance_report.pdf")

# ---------------------
# 5. PPTX 슬라이드 생성 (pptx 설치된 경우만 실행)
# ---------------------
if pptx_available:
    prs = Presentation()

    # 타이틀 슬라이드
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = "배송 성과 분석"
    subtitle.text = "Olist 데이터 기반 경영 보고"

    # KPI 슬라이드
    bullet_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]

    title_shape.text = "핵심 KPI"
    text_frame = body_shape.text_frame
    text_frame.text = f"총 주문 수: {orders_total:,}\n평균 배송일수: {mean_delivery_days:.2f}일\n중앙값 배송일수: {median_delivery_days:.2f}일\n정시배송률: {on_time_rate*100:.2f}%"

    # 리뷰 영향 슬라이드
    slide = prs.slides.add_slide(bullet_slide_layout)
    slide.shapes.title.text = "배송 지연과 리뷰의 관계"
    body_shape = slide.placeholders[1]
    text_frame = body_shape.text_frame
    text_frame.text = f"상관계수: {corr:.3f}\n회귀식: 리뷰 = {a:.3f} * 지연일수 + {b:.3f}"
    slide.shapes.add_picture("delay_vs_review.png", Inches(4), Inches(2), width=Inches(5), height=Inches(3))

    # 주별 성과 슬라이드
    slide = prs.slides.add_slide(bullet_slide_layout)
    slide.shapes.title.text = "주별 배송 성과 (느린 Top 5)"
    rows, cols = state_perf.shape

    left = Inches(0.5)
    top = Inches(2.0)
    width = Inches(9)
    height = Inches(1.0)

    state_table = state_perf.sort_values("mean_delivery_days", ascending=False).head(5)
    table = slide.shapes.add_table(state_table.shape[0]+1, state_table.shape[1], left, top, width, height).table

    # 헤더 채우기
    for j, col_name in enumerate(state_table.columns):
        table.cell(0, j).text = str(col_name)

    # 데이터 채우기
    for i in range(state_table.shape[0]):
        for j in range(state_table.shape[1]):
            table.cell(i+1, j).text = str(round(state_table.iloc[i, j],2))

    # 배송 소요일 분포 슬라이드
    slide = prs.slides.add_slide(bullet_slide_layout)
    slide.shapes.title.text = "배송 소요일 분포"
    slide.shapes.add_picture("delivery_distribution.png", Inches(2), Inches(2), width=Inches(6), height=Inches(4))

    prs.save("shipping_performance_presentation.pptx")
    print("PPTX 프레젠테이션이 생성되었습니다: shipping_performance_presentation.pptx")


⚠️ reportlab 라이브러리가 설치되어 있지 않습니다. 설치하려면 'pip install reportlab'를 실행하세요.


C:\Users\user\AppData\Local\Temp\ipykernel_27188\1434824287.py:69: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1434824287.py:69: UserWarning: Glyph 47928 (\N{HANGUL SYLLABLE MUN}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1434824287.py:69: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1434824287.py:69: UserWarning: Glyph 48176 (\N{HANGUL SYLLABLE BAE}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Local\Temp\ipykernel_27188\1434824287.py:69: UserWarning: Glyph 49569 (\N{HANGUL SYLLABLE SONG}) missing from font(s) DejaVu Sans.
  plt.savefig("delivery_distribution.png")
C:\Users\user\AppData\Loca

TypeError: type str doesn't define __round__ method